# Tutorial: Using Datasets of Source Positions in a Single Plane

In [ ]:
from hartufo import CipicPlane, AriPlane, ListenPlane, CrossModPlane, BiLiPlane, ItaPlane, HutubsPlane, RiecPlane, Sadie2Plane, Princeton3D3APlane, ChedarPlane, WidespreadPlane, ScutPlane, SonicomPlane
import hartufo
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
base_dir = Path('../HRTF Datasets')

## The minimum necessary to get started

The purpose of `hartufo` is to provide a unified programming interface to multiple collections of HRTF data in a way that is convenient and compatible with all major machine learning toolkits. Currently the following data collections are supported:

- CIPIC
- ARI
- Listen
- CrossMod
- BiLi
- ITA
- HUTUBS
- RIEC
- SADIE II
- 3D3A
- CHEDAR
- Widespread
- SCUT
- SONICOM

Each of them has a corresponding class that can be loaded from `hartufo`, as is done above.

At minimum, you need to select which plane to use (out of `horizontal`, `median` or `frontal`), the HRIR/HRTF representation (`time`, `magnitude`, `magnitude_db`, `phase` or `complex`) and the side of the head (`left`, `right`, `both`, `both-left`, `both-right`).

In [ ]:
plane = 'median'
domain = 'magnitude_db'
side = 'left'

These parameters can then be used together with an `XPlane` class and the path to the root directory of files for the corresponding collection (having the same directory structure as on the cluster).

In [ ]:
ds = CipicPlane(base_dir / 'CIPIC', plane, domain, side)

All `XPlane` types are subclasses of `hartufo.Dataset`.

In [ ]:
isinstance(ds, hartufo.Dataset)

You can get the size of a dataset and and access its individual datapoints by indexing it.

In [ ]:
len(ds)

In [ ]:
p = ds[0]

Each datapoint is a dictionary that contains the keys `features`, `target` and `group`.

In [ ]:
p.keys()

The `features` key returns the HRIR values in the specified plane as a matrix. Each row contains a single HRIR for a certain position in the plane, so the number of rows equals the number of measurement positions in the plane and the number of columns is equal to the lenght of an HRIR.

In [ ]:
p['features'].shape

The `target` and `group` keys are empty by default and can be ignored for now (more explanation follows in the next tutorial).

In [ ]:
p['target'], p['group']

You can also get all the HRTFs of all datapoints at once using the `.features` property of the dataset. It returns a 3D array of dimensions: dataset size x number of positions x HRIR length.

In [ ]:
ds.features.shape

This minimal data access functionality should suffice to use the `Dataset` classes in combination with all major machine learning frameworks (see separate tutorials for more info), but a lot more functionality is built-in.

## Additional Dataset Functionality

You can get the sample rate of the HRIR:

In [ ]:
ds.hrir_samplerate

or the frequencies of the HRTF:

In [ ]:
ds.hrtf_frequencies

The angles in the plane can be obtained with:

In [ ]:
ds.plane_angles

By default, the interval for the angles is [-180, 180) (horizontal and frontal plane) or [-90, 27) (median plane). Positive angles in the range [0, 360) can be requested by passing `positive_angles=True` to the `XPlane` constructor.

The angle interval can also be changed after the creation of the dataset, without needing to reload from disk, by setting the `.positive_angles` boolean property.

In [ ]:
ds.positive_angles = True
ds.plane_angles

In all cases, the angle extrema are available as:

In [ ]:
ds.min_angle, ds.max_angle

and the name of the angle in the plane can be obtained by:

In [ ]:
ds.plane_angle_name

Using this additional info, you could create your own plots, but plotting functionality is also included. You can plot the angles of the plane with the `.plot_angles()` method.

In [ ]:
ds.positive_angles = False
ds.plot_angles()
plt.show()

Its complete type signature is `.plot_angles(ax=None, title=None)`, which contains optional arguments to draw on existing Matplotlib `Axes` or to overwrite the default title.

You can also plot the HRTF of a data point, by passing its index to `.plot_plane()`.

In [ ]:
ds.plot_plane(0)
plt.show()

The type signature of this method is `.plot_plane(idx, ax=None, cmap='viridis', continuous=False, vmin=None, vmax=None, title=None, colorbar=True, log_freq=False)`, which contains quite some optional arguments. The `ax` and `title` options do the same as in `.plot_angles()`. Any [colour map available in Matplotlib](https://matplotlib.org/stable/gallery/color/colormap_reference.html) can be used by passing it to `cmap`. By default the minimum and maximum values of the colour map are set to the minimum and maximum value in the entire dataset, but they can be set using the `vmin` and `vmax` options. The colour bar that is shown by default can be disabled with `colorbar=False`.

The plane angles are plotted on a linear scale, so if the sampling of angles is non-uniform, certain angles will be drawn over larger areas in the plot than others. By default, the area up to halfway the next angle is filled with a uniform colour, resulting in a block-like appearance that can be used to visually inspect the distribution of angles in the plane. By passing `continuous=True`, intermediate angle values will be interpolated leading to a smooth picture. For frequency-domain HRTF representations, the option `log_freq=True` can be used to plot frequency on a logarithmic axis. Finally, the Matplotlib `Axes` get returned, allowing for further customisation of the plot.

An example demonstrating all these options can be found below.

In [ ]:
_, ax = plt.subplots(figsize=(8,8))
ax = ds.plot_plane(0, ax=ax, cmap='gray', continuous=True, vmin=-120, vmax=0, title='', colorbar=False, log_freq=True)
ax.set_ylim(0.1, 18)
plt.show()

## Customising Dataset Contents

### Selecting specific subjects

By default, all available subjects will be loaded into a dataset. A list of ids of the subjects composing the dataset can be obtained as:

In [ ]:
ds.subject_ids

In order to make a selection of the subjects, you can pass the `subject_ids` argument to an `XPlane` constructor. It should contain an iterable of subject ids.

In [ ]:
ds = CipicPlane(base_dir / 'CIPIC', plane, domain=domain, side=side, subject_ids=(1, 2, 3, 4, 5))

If no subject with the given id exists, it gets silently skipped.

In [ ]:
ds.subject_ids

For any dataset, regardless its contents, you can request the possible subject ids you can pass to the constructor:

In [ ]:
ds.available_subject_ids

This makes the following workflow a convenient way to split a dataset into parts. Start by creating an empty data set by passing an empty list or tuple to `subject_ids`, then read `.available_subject_ids` to find out what subjects are available and create another data set with a subset of these ids.

In [ ]:
ds = AriPlane(base_dir / 'ARI', plane, domain=domain, side=side, subject_ids=())
ds.subject_ids

If you just need a single example of a data collection, you can instead pass one of the strings `first`, `last` or `random` to `subject_ids`. The first two deterministically load the first, respectively last, id in the collection, whereas `random` loads a random subject.

In [ ]:
ds = AriPlane(base_dir / 'ARI', plane, domain=domain, side=side, subject_ids='first')
ds.subject_ids

In [ ]:
ds = AriPlane(base_dir / 'ARI', plane, domain=domain, side=side, subject_ids='last')
ds.subject_ids

In [ ]:
ds = AriPlane(base_dir / 'ARI', plane, domain=domain, side=side, subject_ids='random')
ds.subject_ids

### Selecting specific measurement positions

Likewise, instead of loading all measurement positions in the plane, a selection can be made by passing a `plane_angles` argument to the constructor.

In [ ]:
ds = AriPlane(base_dir / 'ARI', plane, domain=domain, side=side, subject_ids='random', plane_angles=(0, 3.14, 10, 20, 30))
ds.plane_angles

Missing plane angles get silently skipped as well.

If only a small number of measurement angles are selected, an alternative visualisation using a lineplot instead of a heatmap can be produced by passing `lineplot=True` to the `.plot_plane()` method.

In [ ]:
ds.plot_plane(0, lineplot=True)

### Customising numeric precision

By default, the HRIR/HRTF values get stored as `np.float32`s. You can however change it by passing a `dtype` argument to the `XPlane` constructor.

In [ ]:
ds = AriPlane(base_dir / 'ARI', plane, domain=domain, side=side, subject_ids='first', dtype=np.float64)
ds[0]['features'].dtype

When requesting an HRTF in the `complex` domain, it is necessary to specify a complex dtype, otherwise an error will be thrown.

In [ ]:
try:
    ds = AriPlane(base_dir / 'ARI', plane, domain='complex', side=side, subject_ids='first')
except ValueError as e:
    print(e)

In [ ]:
ds = AriPlane(base_dir / 'ARI', plane, domain='complex', side=side, subject_ids='first', dtype=np.complex64)
ds[0]['features'].dtype

## Illustrative example

As a concluding example, the snippet below plots the angular distribution and an example HRTF of all available data collections for each of the three principal planes.

In [ ]:
domain = 'magnitude_db'
side = 'left'
positive_angles = False
subject_ids = 'first'

for collection, data_dir in [
    (CipicPlane, base_dir / 'CIPIC'),
    (AriPlane, base_dir / 'ARI'),
    (ListenPlane, base_dir / 'Ircam Listen'),
    (CrossModPlane, base_dir / 'Ircam CrossMod'),
    (BiLiPlane, base_dir / 'Ircam BiLi'),
    (ItaPlane, base_dir / 'ITA'), 
    (HutubsPlane, base_dir / 'HUTUBS'),
    (RiecPlane, base_dir / 'RIEC'),
    (Sadie2Plane, base_dir / 'SADIE II'),
    (Princeton3D3APlane, base_dir / '3D3A'),
    (ChedarPlane, base_dir / 'CHEDAR'),
    (WidespreadPlane, base_dir / 'Widespread'),
    (ScutPlane, base_dir / 'SCUT'),
    (SonicomPlane, base_dir / 'SONICOM'),
]:
    fig = plt.figure(figsize=(16, 18))
    fig.suptitle(collection.__name__)
    for idx, plane in enumerate(('horizontal', 'median', 'frontal')):
        plane_offset = -0.72 if collection == ItaPlane and plane == 'horizontal' else 0
        ds = collection(data_dir, plane, plane_offset=plane_offset, domain=domain, side=side, positive_angles=positive_angles, subject_ids=subject_ids)
        ax0 = fig.add_subplot(3, 2, 2*idx+1, projection='polar')
        ax1 = fig.add_subplot(3, 2, 2*idx+2)
        if domain.startswith('magnitude'):
            ax1.set_ylim((0, 18))
        elif domain == 'time':
            ax1.set_ylim((0, 3))
        ds.plot_angles(ax=ax0)
        ds.plot_plane(0, ax=ax1, continuous=False, log_freq=False)
    plt.show()